# <center>  Deep Neural Network (DNN) using plain TensorFlow

## <font color = 'blue'> Get the data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from collections import defaultdict
from time import time

%store -r
%store

In [2]:
print("Training data: ", X_32train_std.shape, "and labels: ", y_32_train.shape)
print(" ")
print("Test data:")
print("Total", len(X_32test_std), X_32test_std[18].shape, "arrays for SNR values:")
print(sorted(X_32test_std.keys()))

Training data:  (80000, 32) and labels:  (80000,)
 
Test data:
Total 20 (4000, 32) arrays for SNR values:
[-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [3]:
# tf.reset_default_graph()

## <font color='blue'> Create validation set required for early stopping

In [4]:
X_val = []
y_val = []

X_test_new = defaultdict(list)
y_test_new = defaultdict(list)

for snr in snrs:
    n_test = X_test_std[snr].shape[0]
    X_val.append(X_test_std[snr][:(n_test*0.5)])
    y_val.append(y_test[snr][:(n_test*0.5)])
    X_test_new[snr] = X_test_std[snr][(n_test*0.5):]
    y_test_new[snr] = y_test[snr][(n_test*0.5):]
    
X_val = np.vstack(np.asarray(X_val))
y_val =np.hstack(np.asarray(y_val))

print("Validation set data", X_val.shape, "and labels", y_val.shape)

X_test_std = X_test_new
y_test = y_test_new

Validation set data (40000, 16) and labels (40000,)


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


## <font color = 'blue'> Design and train the DNN

In [5]:
# ----------------- Initialize parameters -----------------------


n_hidden1 = 32
n_hidden2 = 32
n_outputs = 8

X = tf.placeholder(tf.float32, shape=(None, 32))

y = tf.placeholder(tf.int64, shape=(None))

# ------------------- Define layers -----------------------

from tensorflow.contrib.layers import fully_connected

def layer(X, n_neurons, activation):
    layer = fully_connected(X, n_neurons, activation_fn = activation, 
                            weights_initializer = tf.contrib.layers.variance_scaling_initializer())
    return layer

layer1 = layer(X, n_hidden1, activation = tf.nn.elu)

layer2 = layer(layer1, n_hidden2, activation = tf.nn.elu)

logits = layer(layer2, n_outputs, activation = None)

# ----------------- Specify performance measure ----------------------

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)

optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        
# ------------------ Execution phase ----------------------------------    

n_epochs = 1000
batch_size = 1024
n_train = X_train_std.shape[0]
n_iter = n_train//batch_size

saver = tf.train.Saver()

min_loss = np.infty
epochs_without_improvement = 0 
max_epochs_without_improvement = 20   

acc_test = defaultdict(list)

start = time()
path = "./DNN32_elu_xavier.ckpt"

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        for iteration in range(n_iter):
            rand_indices = np.random.choice(n_train,batch_size) #select random samples to form mini batches   
            X_batch, y_batch = X_32train_std[rand_indices], y_32_train[rand_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        valid_loss, acc_val = sess.run([loss, accuracy], feed_dict={X: X_32_val, y: y_32_val})
        
        # Early stopping 
        
        if valid_loss < min_loss:
            save_path = saver.save(sess, path)
            min_loss = valid_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement > max_epochs_without_improvement:
                print("** EARLY STOPPING ** ")
                break
        print("{}\tValidation loss: {:.6f}\tMinimum loss: {:.6f}\tAccuracy on validation set: {:.2f}".format(
            epoch, valid_loss, min_loss, acc_val))

print(" ")
with tf.Session() as sess:
    saver.restore(sess, path)
    for snr in snrs:
        acc_test[snr] = accuracy.eval(feed_dict={X: X_32test_std[snr], y: y_32_test[snr]})

print("Training and testing took %f minutes"%(float(time() - start)/60))

0	Validation loss: 1.751243	Minimum loss: 1.751243	Accuracy on validation set: 0.31
1	Validation loss: 1.623884	Minimum loss: 1.623884	Accuracy on validation set: 0.36
2	Validation loss: 1.563519	Minimum loss: 1.563519	Accuracy on validation set: 0.38
3	Validation loss: 1.522553	Minimum loss: 1.522553	Accuracy on validation set: 0.39
4	Validation loss: 1.489331	Minimum loss: 1.489331	Accuracy on validation set: 0.40
5	Validation loss: 1.461393	Minimum loss: 1.461393	Accuracy on validation set: 0.41
6	Validation loss: 1.437970	Minimum loss: 1.437970	Accuracy on validation set: 0.42
7	Validation loss: 1.419407	Minimum loss: 1.419407	Accuracy on validation set: 0.43
8	Validation loss: 1.401177	Minimum loss: 1.401177	Accuracy on validation set: 0.43
9	Validation loss: 1.386079	Minimum loss: 1.386079	Accuracy on validation set: 0.43
10	Validation loss: 1.372277	Minimum loss: 1.372277	Accuracy on validation set: 0.44
11	Validation loss: 1.362826	Minimum loss: 1.362826	Accuracy on validation 

## <font color = 'blue'> Test the DNN

In [6]:
for snr in snrs:
    print("DNN's test accuracy on %d dB SNR samples = "%(snr), acc_test[snr])   

DNN's test accuracy on -20 dB SNR samples =  0.12225
DNN's test accuracy on -18 dB SNR samples =  0.13025
DNN's test accuracy on -16 dB SNR samples =  0.1285
DNN's test accuracy on -14 dB SNR samples =  0.12475
DNN's test accuracy on -12 dB SNR samples =  0.1505
DNN's test accuracy on -10 dB SNR samples =  0.18975
DNN's test accuracy on -8 dB SNR samples =  0.301
DNN's test accuracy on -6 dB SNR samples =  0.3675
DNN's test accuracy on -4 dB SNR samples =  0.4085
DNN's test accuracy on -2 dB SNR samples =  0.48225
DNN's test accuracy on 0 dB SNR samples =  0.57075
DNN's test accuracy on 2 dB SNR samples =  0.6985
DNN's test accuracy on 4 dB SNR samples =  0.813
DNN's test accuracy on 6 dB SNR samples =  0.8405
DNN's test accuracy on 8 dB SNR samples =  0.84075
DNN's test accuracy on 10 dB SNR samples =  0.844
DNN's test accuracy on 12 dB SNR samples =  0.84175
DNN's test accuracy on 14 dB SNR samples =  0.84125
DNN's test accuracy on 16 dB SNR samples =  0.8335
DNN's test accuracy on 1

## <font color = 'blue'> Visualize DNN's performance

In [7]:
import matplotlib.pyplot as plt

plt.style.use('classic')

fig = plt.figure(figsize=(8, 4), dpi=100)
x = snrs
y = list(acc_test.values())
plt.plot(x, y, marker="o", linewidth=2.0, linestyle='dashed', color='royalblue')
plt.axis([-20, 20, 0, 1])
plt.xticks(np.arange(min(x), max(x)+1, 2.0))
plt.yticks(np.arange(0, 1, 0.10))

ttl = plt.title('SNR vs Accuracy', fontsize=16)
ttl.set_weight('bold')
plt.xlabel('SNR (dB)', fontsize=14)
plt.ylabel('Test accuracy', fontsize=14)
plt.grid()

plt.show()

In [8]:
acc_test

defaultdict(list,
            {-20: 0.12225,
             -18: 0.13025001,
             -16: 0.1285,
             -14: 0.12475,
             -12: 0.1505,
             -10: 0.18975,
             -8: 0.301,
             -6: 0.36750001,
             -4: 0.40849999,
             -2: 0.48225001,
             0: 0.57075,
             2: 0.69849998,
             4: 0.81300002,
             6: 0.8405,
             8: 0.84074998,
             10: 0.84399998,
             12: 0.84175003,
             14: 0.84125,
             16: 0.83350003,
             18: 0.84399998})